# Al Karama Urban Climate Analysis - Code Guide

This notebook demonstrates the Python code and libraries used for each analysis.

## Table of Contents
1. [Library Overview](#1-library-overview)
2. [Street View Analysis (ZenSVI)](#2-street-view-analysis)
3. [Satellite Analysis (Google Earth Engine)](#3-satellite-analysis)
4. [Network Analysis (OSMnx + NetworkX)](#4-network-analysis)
5. [Cluster Analysis (scikit-learn)](#5-cluster-analysis)
6. [Visualization (Folium/Leaflet)](#6-visualization)

---
## 1. Library Overview

### Required Libraries by Analysis Type

| Analysis | Libraries | Install Command |
|----------|-----------|----------------|
| Street View (GVI, SVF) | `zensvi` | `pip install zensvi` |
| Satellite (LST, NDVI) | `earthengine-api` | `pip install earthengine-api` |
| Network (Centrality) | `osmnx`, `networkx` | `pip install osmnx networkx` |
| Clustering | `scikit-learn` | `pip install scikit-learn` |
| Spatial Data | `geopandas`, `shapely` | `pip install geopandas shapely` |
| Visualization | `folium`, `matplotlib` | `pip install folium matplotlib` |
| Data Processing | `pandas`, `numpy`, `scipy` | `pip install pandas numpy scipy` |

In [ ]:
# Install all required libraries (run once)
# !pip install zensvi earthengine-api osmnx networkx scikit-learn geopandas folium pandas numpy scipy

In [ ]:
# Check installed versions
import pkg_resources

packages = ['zensvi', 'earthengine-api', 'osmnx', 'networkx', 
            'scikit-learn', 'geopandas', 'folium', 'pandas', 'numpy']

print("Installed Package Versions:")
print("-" * 40)
for pkg in packages:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg:20s} {version}")
    except:
        print(f"{pkg:20s} NOT INSTALLED")

---
## 2. Street View Analysis

### Libraries Required
```python
from zensvi.download import MLYDownloader  # Download Mapillary images
from zensvi.cv import ClassifierGVI        # Calculate Green View Index
from zensvi.cv import ClassifierSVF        # Calculate Sky View Factor
```

### 2.1 Download Street View Images from Mapillary

In [ ]:
from zensvi.download import MLYDownloader

# Initialize downloader
mly = MLYDownloader(log_path="logs")

# Download images within a GeoJSON boundary
mly.download_svi(
    dir_output="data/mapillary_svi",      # Output directory
    input_shp="data/al_karama.geojson",   # Study area boundary
    lat=None,                              # Or specify lat/lon for point
    lon=None,
    id_columns=None
)

# Note: Requires Mapillary API token in environment variable MLY_TOKEN

### 2.2 Calculate Green View Index (GVI)

In [ ]:
from zensvi.cv import ClassifierGVI

# Initialize GVI classifier
gvi_classifier = ClassifierGVI()

# Calculate GVI for all images in a directory
gvi_classifier.classify(
    dir_input="data/mapillary_svi",       # Input image directory
    dir_output="output/gvi_results",       # Output directory
    csv_output="output/gvi_results.csv"    # Results CSV
)

# GVI is calculated as:
# GVI = (green pixels) / (total pixels) * 100%
# Uses vegetation detection based on color analysis

### 2.3 Calculate Sky View Factor (SVF)

In [ ]:
from zensvi.cv import ClassifierSVF

# Initialize SVF classifier  
svf_classifier = ClassifierSVF()

# Calculate SVF for all images
svf_classifier.classify(
    dir_input="data/mapillary_svi",
    dir_output="output/svf_results",
    csv_output="output/svf_results.csv"
)

# SVF is calculated as:
# SVF = (sky pixels) / (upper hemisphere pixels) * 100%
# Lower SVF = more shade from buildings/trees

---
## 3. Satellite Analysis

### Libraries Required
```python
import ee  # Google Earth Engine
```

### 3.1 Initialize Google Earth Engine

In [ ]:
import ee

# First time: Authenticate (opens browser)
# ee.Authenticate()

# Initialize with your project
ee.Initialize(project='your-project-id')  # Replace with your GEE project

print("Google Earth Engine initialized successfully!")

### 3.2 Define Study Area

In [ ]:
# Define bounding box for Al Karama
north, south = 25.255, 25.230
east, west = 55.315, 55.290

# Create Earth Engine geometry
study_area = ee.Geometry.Rectangle([west, south, east, north])

print(f"Study area: {(north-south)*111:.1f}km x {(east-west)*111:.1f}km")

### 3.3 Calculate Land Surface Temperature (LST) from Landsat

In [ ]:
def calculate_lst(image):
    """
    Calculate Land Surface Temperature from Landsat 8/9 thermal band.
    
    Formula:
    LST = BT / (1 + (λ × BT / ρ) × ln(ε))
    
    Where:
    - BT = Brightness Temperature
    - λ = wavelength (10.8 μm for Landsat Band 10)
    - ρ = h × c / σ = 14380 μm·K
    - ε = emissivity (0.95 for urban)
    """
    # Get thermal band and convert to brightness temperature
    thermal = image.select('ST_B10').multiply(0.00341802).add(149.0)
    
    # Constants
    wavelength = 10.8  # μm
    rho = 14380        # μm·K
    emissivity = 0.95  # urban surfaces
    
    # Calculate LST in Celsius
    lst = thermal.divide(
        ee.Number(1).add(
            ee.Number(wavelength).multiply(thermal).divide(rho)
            .multiply(ee.Number(emissivity).log())
        )
    ).subtract(273.15)  # Convert Kelvin to Celsius
    
    return image.addBands(lst.rename('LST'))

# Load Landsat 8/9 Collection 2
landsat = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(study_area) \
    .filterDate('2024-06-01', '2024-08-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(calculate_lst)

# Get median LST
lst_image = landsat.select('LST').median().clip(study_area)

print(f"Landsat images found: {landsat.size().getInfo()}")

### 3.4 Calculate NDVI from Sentinel-2

In [ ]:
def calculate_indices(image):
    """
    Calculate vegetation and built-up indices from Sentinel-2.
    
    NDVI = (NIR - Red) / (NIR + Red)
    NDBI = (SWIR - NIR) / (SWIR + NIR)
    """
    # NDVI: Normalized Difference Vegetation Index
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # NDBI: Normalized Difference Built-up Index
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    
    # NDWI: Normalized Difference Water Index
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    
    return image.addBands([ndvi, ndbi, ndwi])

# Load Sentinel-2 Surface Reflectance
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(study_area) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(calculate_indices)

# Get median indices
indices = sentinel2.select(['NDVI', 'NDBI', 'NDWI']).median().clip(study_area)

print(f"Sentinel-2 images found: {sentinel2.size().getInfo()}")

### 3.5 Sample Data at Grid Points

In [ ]:
import pandas as pd
import numpy as np

def sample_satellite_data(combined_image, study_area, grid_spacing=0.0003):
    """
    Sample satellite data at regular grid points.
    
    Args:
        combined_image: ee.Image with bands to sample
        study_area: ee.Geometry boundary
        grid_spacing: degrees (~30m at equator)
    """
    # Create grid points
    bounds = study_area.bounds().getInfo()['coordinates'][0]
    min_lon, min_lat = bounds[0]
    max_lon, max_lat = bounds[2]
    
    lats = np.arange(min_lat, max_lat, grid_spacing)
    lons = np.arange(min_lon, max_lon, grid_spacing)
    
    # Create feature collection of points
    points = []
    for lat in lats:
        for lon in lons:
            points.append(ee.Feature(ee.Geometry.Point([lon, lat])))
    
    fc = ee.FeatureCollection(points)
    
    # Sample the image
    sampled = combined_image.sampleRegions(
        collection=fc,
        scale=10,  # 10m resolution
        geometries=True
    )
    
    return sampled

# Combine LST and indices into single image
combined = lst_image.addBands(indices)

# Sample (for small areas - for large areas, use batch processing)
# sampled_data = sample_satellite_data(combined, study_area)

---
## 4. Network Analysis

### Libraries Required
```python
import osmnx as ox      # Download OpenStreetMap street networks
import networkx as nx   # Graph analysis
```

### 4.1 Download Street Network from OpenStreetMap

In [ ]:
import osmnx as ox
import networkx as nx

# Define bounding box
north, south = 25.255, 25.230
east, west = 55.315, 55.290

# Download walking network
G = ox.graph_from_bbox(
    bbox=(north, south, east, west),
    network_type='walk'  # Options: 'walk', 'drive', 'bike', 'all'
)

print(f"Nodes (intersections): {G.number_of_nodes()}")
print(f"Edges (street segments): {G.number_of_edges()}")

# Basic stats
stats = ox.stats.basic_stats(G)
print(f"Total street length: {stats['street_length_total']/1000:.1f} km")

### 4.2 Calculate Betweenness Centrality

In [ ]:
# Convert to undirected for centrality analysis
G_undir = G.to_undirected()

# Calculate node betweenness centrality
# This measures how often a node lies on shortest paths between other nodes
node_betweenness = nx.betweenness_centrality(
    G_undir, 
    weight='length',    # Use street length as weight
    normalized=True     # Normalize to 0-1
)

# Add to graph as node attribute
nx.set_node_attributes(G_undir, node_betweenness, 'betweenness')

# Find top 5 most central nodes
top_nodes = sorted(node_betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 nodes by betweenness centrality:")
for node_id, bc in top_nodes:
    print(f"  Node {node_id}: {bc:.4f}")

### 4.3 Calculate Closeness Centrality

In [ ]:
# Calculate closeness centrality
# This measures how close a node is to all other nodes
node_closeness = nx.closeness_centrality(
    G_undir,
    distance='length'  # Use street length as distance
)

# Add to graph
nx.set_node_attributes(G_undir, node_closeness, 'closeness')

print(f"Closeness centrality - Mean: {np.mean(list(node_closeness.values())):.6f}")
print(f"Closeness centrality - Max: {max(node_closeness.values()):.6f}")

### 4.4 Convert to GeoDataFrame for Spatial Analysis

In [ ]:
# Convert graph to GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_undir)

print(f"Nodes GeoDataFrame: {len(nodes_gdf)} rows")
print(f"Edges GeoDataFrame: {len(edges_gdf)} rows")

# Nodes now have betweenness and closeness as columns
print("\nNode columns:", nodes_gdf.columns.tolist())

---
## 5. Cluster Analysis

### Libraries Required
```python
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
```

### 5.1 Prepare Data for Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load your combined data
df = pd.read_csv('output/walking_routes/point_comfort.csv')

# Select features for clustering
feature_cols = ['gvi', 'svf', 'lst', 'ndvi']
X = df[feature_cols].dropna()

print(f"Data points for clustering: {len(X)}")
print(f"Features: {feature_cols}")

### 5.2 Standardize Features

In [ ]:
# Standardize features (important for K-means!)
# This converts all features to z-scores (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Before scaling:")
print(f"  GVI range: {X['gvi'].min():.3f} - {X['gvi'].max():.3f}")
print(f"  LST range: {X['lst'].min():.1f} - {X['lst'].max():.1f}")

print("\nAfter scaling (all features now comparable):")
print(f"  All features: mean ≈ 0, std ≈ 1")

### 5.3 Find Optimal Number of Clusters (Elbow Method)

In [ ]:
# Elbow method to find optimal k
inertias = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia (within-cluster sum of squares)')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

print("Look for the 'elbow' where the curve bends - that's the optimal k")

### 5.4 Run K-Means Clustering

In [ ]:
# Run K-means with chosen k
n_clusters = 4

kmeans = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10  # Number of initializations
)

# Fit and predict
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataframe
X['cluster'] = clusters

# Analyze clusters
print("Cluster characteristics:")
print("-" * 60)
for i in range(n_clusters):
    cluster_data = X[X['cluster'] == i]
    print(f"\nCluster {i} ({len(cluster_data)} points):")
    print(f"  GVI:  {cluster_data['gvi'].mean()*100:.1f}%")
    print(f"  SVF:  {cluster_data['svf'].mean()*100:.1f}%")
    print(f"  LST:  {cluster_data['lst'].mean():.1f}°C")
    print(f"  NDVI: {cluster_data['ndvi'].mean():.3f}")

---
## 6. Visualization

### Libraries Required
```python
import folium           # Interactive maps
import matplotlib.pyplot as plt  # Static plots
```

### 6.1 Create Interactive Map with Folium

In [ ]:
import folium

# Create base map centered on Al Karama
m = folium.Map(
    location=[25.2425, 55.3025],  # Center coordinates
    zoom_start=15,
    tiles='OpenStreetMap'
)

# Add circle markers for each point
for idx, row in df.head(100).iterrows():  # Limit for demo
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color='blue',
        fill=True,
        fillOpacity=0.7,
        popup=f"GVI: {row['gvi']*100:.1f}%<br>LST: {row['lst']:.1f}°C"
    ).add_to(m)

# Display map (in Jupyter)
m

### 6.2 Color Points by Value

In [ ]:
def get_color(value, min_val, max_val, colormap='RdYlGn'):
    """
    Convert a value to a color based on a colormap.
    """
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    
    # Normalize value to 0-1
    norm = (value - min_val) / (max_val - min_val)
    norm = max(0, min(1, norm))  # Clamp to 0-1
    
    # Get color from colormap
    cmap = cm.get_cmap(colormap)
    rgba = cmap(norm)
    
    # Convert to hex
    return colors.rgb2hex(rgba)

# Create map with colored points by LST
m2 = folium.Map(location=[25.2425, 55.3025], zoom_start=15)

lst_min, lst_max = df['lst'].min(), df['lst'].max()

for idx, row in df.head(500).iterrows():
    color = get_color(row['lst'], lst_min, lst_max, 'RdYlGn_r')  # Red=hot
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m2)

m2

### 6.3 Create Correlation Heatmap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate correlation matrix
corr_cols = ['gvi', 'svf', 'lst', 'ndvi', 'ndbi']
corr_matrix = df[corr_cols].corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    corr_matrix,
    annot=True,           # Show values
    cmap='RdBu_r',        # Red-Blue colormap
    center=0,             # Center at 0
    vmin=-1, vmax=1,      # Range -1 to 1
    square=True,
    fmt='.2f'             # 2 decimal places
)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

---
## 7. Putting It All Together

### Complete Workflow Example

In [ ]:
# =============================================================
# COMPLETE WORKFLOW SUMMARY
# =============================================================

# Step 1: Download street view images
# from zensvi.download import MLYDownloader
# mly = MLYDownloader()
# mly.download_svi(dir_output="data/svi", input_shp="boundary.geojson")

# Step 2: Calculate GVI and SVF
# from zensvi.cv import ClassifierGVI, ClassifierSVF
# ClassifierGVI().classify(dir_input="data/svi", csv_output="gvi.csv")
# ClassifierSVF().classify(dir_input="data/svi", csv_output="svf.csv")

# Step 3: Get satellite data from Google Earth Engine
# import ee
# ee.Initialize(project='your-project')
# landsat = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')...
# sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')...

# Step 4: Download street network
# import osmnx as ox
# G = ox.graph_from_bbox(bbox=(...), network_type='walk')

# Step 5: Calculate centrality
# import networkx as nx
# betweenness = nx.betweenness_centrality(G, weight='length')
# closeness = nx.closeness_centrality(G, distance='length')

# Step 6: Combine data and analyze
# import pandas as pd
# df = pd.merge(gvi_df, svf_df, on='id')
# df = spatial_join(df, satellite_df)  # Join by location

# Step 7: Calculate priority scores
# df['priority'] = 0.4 * df['lst_norm'] + 0.25 * (1 - df['gvi_norm']) + ...

# Step 8: Cluster analysis
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=4)
# df['cluster'] = kmeans.fit_predict(X_scaled)

# Step 9: Visualize
# import folium
# m = folium.Map(location=[lat, lon])
# ...

print("See the individual sections above for detailed code examples!")

---
## References

### Libraries Documentation
- ZenSVI: https://github.com/koito19960406/ZenSVI
- Google Earth Engine: https://developers.google.com/earth-engine
- OSMnx: https://osmnx.readthedocs.io/
- NetworkX: https://networkx.org/documentation/
- scikit-learn: https://scikit-learn.org/stable/
- Folium: https://python-visualization.github.io/folium/

### Academic References
- GVI: Yang et al. (2009) "The urban forest in Beijing and its role in air pollution reduction"
- SVF: Oke (1981) "Canyon geometry and the nocturnal urban heat island"
- NDVI: Tucker (1979) "Red and photographic infrared linear combinations for monitoring vegetation"
- Betweenness Centrality: Freeman (1977) "A set of measures of centrality based on betweenness"